In [ ]:
import numpy as np
import h5py
import cv2
import glob
import matplotlib.pyplot as plt
import time
import matplotlib.patches as patches

# The bytes file version

In [ ]:
def visualise_goal_vs_path(start,goal,poses,legend_pos="upper right"):
    '''
    Visualise the goal path and the path taken
    
    
    '''

    start_x=start[0]
    start_y=start[1]    
    goal_x=goal[0]
    goal_y=goal[1]
    path_x=poses[:,0]
    path_y=poses[:,1]
    # plt.plot(start_x,start_y,'ro',label="start")
    # plt.plot(goal_x,goal_y,'go',label="goal")
    # plt.Circle((0, 0), 0.2, color='r')
    # plt.plot(path_x,path_y,'-b',label="path")
    # plt.xlabel('X coordinate (m)')
    # plt.ylabel('Y coordinate (m)')
    # plt.axis('equal')
    # plt.grid()
    # plt.title('Goal vs Path Relative to Starting Position')
    # plt.legend(loc=legend_pos)
    # plt.show()

    fig, ax = plt.subplots()
    circle = patches.Circle((goal_x, goal_y),1.5, edgecolor='blue', facecolor='none', linewidth=2)
    ax.add_patch(circle)
    ax.plot(start_x,start_y,'ro',label="start")
    ax.plot(goal_x,goal_y,'go',label="goal")
    ax.plot(path_x,path_y,'-b',label="path")
    ax.set_xlabel('X coordinate (m)')
    ax.set_ylabel('Y coordinate (m)')
    plt.axis('equal')
    plt.grid()
    plt.title('Goal vs Path Relative to Starting Position')
    plt.legend(loc=legend_pos)
    plt.show()



def visualise_target_vs_iters(targets):
    '''
    Visualise the goal path and the path taken
    
    
    '''
    abs_targets=np.ones(shape=(len(targets),1))
    abs_targets[:,0]=np.square(np.sum(targets[:,0:1],axis=1))   
    abs_targets[:,0]=np.sqrt(abs_targets[:,0])   
    plt.plot(abs_targets,'-b',label="path")
    plt.xlabel('Time (s)')
    plt.ylabel('Radial distance from goal (m)')
    plt.title('Radial distance from goal vs time')
    plt.grid()
    plt.show()


def visualise_comaparison(start,goal,expert_traj,agent_traj,legend_pos="upper right"):
    '''
    Comaprea expert and agent traj
    
    '''

    start_x=start[0]
    start_y=start[1]    
    goal_x=goal[0]
    goal_y=goal[1]
    path_x=expert_traj[:,0]
    path_y=expert_traj[:,1]
    path_x_agent=agent_traj[:,0]
    path_y_agent=agent_traj[:,1]
    plt.plot(start_x,start_y,'ro',label="start")
    plt.plot(goal_x,goal_y,'go',label="goal")
    plt.plot(path_x,path_y,'-b',label="path of expert")
    plt.plot(path_x_agent,path_y_agent,'-m',label="path of agent")
    plt.xlabel('X coordinate (m)')
    plt.ylabel('Y coordinate (m)')
    plt.grid()
    plt.axis('equal')
    plt.title('Path Taken by Expert vs Agent Relative to Starting Position')
    plt.legend(loc=legend_pos)
    plt.show()

def read_dataset(file_path):
    '''
    Read the data files from the dataset
    '''
    read_file= h5py.File(file_path, "r")
    batch_size= read_file['kris_dynamics']['odom_data']['target_vector'].shape[0]
    rgbs=[]
    depths=[]
    targets=[]  
    acts=[]
    for i in range(batch_size):
        target=read_file['kris_dynamics']['odom_data']['target_vector'][i]
        rgb=read_file['images']['rgb_data'][i]
        depth=read_file['images']['depth_data'][i]
        act=read_file['kris_dynamics']['odom_data']['odom_data_wheel'][i]    
        ## If visualisation is needed
        # if visualise_img:
        #     visualise(rgb_features,depth_features)  
        rgbs.append(rgb)
        depths.append(depth)
        targets.append(target) 
        acts.append(act)

    acts=np.array(acts[:-1])
    rgbs=np.array(rgbs)
    depths=np.array(depths)
    targets=np.array(targets)

    return acts,targets

In [ ]:
# File paths
expert_file='../GailNavigationNetwork/data/trajectories/tests/hard_world/traj1.hdf5'
agent_file='../GailNavigationNetwork/data/trajectories/tests/hard_world/test2.hdf5'

#Variables of expert
acts_expert,targets_expert=read_dataset(expert_file)
acts_expert=acts_expert[:,:2]
targets_expert=targets_expert[:,:2]
print(f"acts shape is {acts_expert.shape} first one is {acts_expert[0]} \n shape is {targets_expert.shape} first one is {targets_expert[0]}")
starting_pose=acts_expert[0]
goal_pose=starting_pose+targets_expert[0]
print(f"Demo Starting pose is {starting_pose} and goal_pose is {goal_pose}")

#Variables of agent

acts_agent,targets_agent=read_dataset(agent_file)
acts_agent=acts_agent[:,:2]
targets_agent=targets_agent[:,:2]
print(f"acts shape is {acts_agent.shape} first one is {acts_agent[0]} \n shape is {targets_agent.shape} first one is {targets_agent[0]}")


In [ ]:
visualise_goal_vs_path(starting_pose,goal_pose,acts_expert,"upper left")

In [ ]:
visualise_target_vs_iters(targets=acts_expert)

In [ ]:
# for i in range(len(targets_demo)):
#     print(f"target is {targets_demo[i]} \n acts is {acts_demo}")
#     time.sleep(0.5)

In [ ]:
visualise_comaparison(starting_pose,goal_pose,acts_expert,acts_agent,"upper left")